In [1]:
import numpy as np
import xarray as xr
import dask
import xesmf as xe
import os
from glob import glob

### Preliminaries

In [7]:
############
# Dask
############
from dask_jobqueue import PBSCluster
cluster = PBSCluster(cores=1, resource_spec='pmem=20GB', memory='20GB',
                     env_extra= ['#PBS -l feature=rhel7'], walltime = '00:20:00')

cluster.scale(jobs=10)  # ask for jobs

from dask.distributed import Client
client = Client(cluster)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.102.201.239:43067,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## ISIMIP

In [ ]:
# Models
from utils import isimip_ssp_dict

models = list(isimip_ssp_dict.keys())

In [8]:
# Set path
# UPDATE THIS FOR REPRODUCTION
path = '/gpfs/group/kaf26/default/dcl5300/lafferty-sriver_inprep_tbh_DATA/metrics/isimip3b/'

In [8]:
# NEX-GDDP out grid (same as CIL)
out_grid = xr.open_dataset('/gpfs/group/kaf26/default/dcl5300/lafferty-sriver_inprep_tbh_DATA/metrics/nex-gddp/annual_avgs/CanESM5.nc')

out_grid = xr.Dataset({'lat': out_grid.lat,
                       'lon': out_grid.lon})

In [11]:
# Regridding function
def regrid(model, out_grid, out_path, metric):
    # Read native grid
    ds = xr.open_dataset(out_path + 'native_grid/' + metric + '/' + model + '.nc')
    
    ## xESMF regridder
    # Billinear
    blnr_regridder = xe.Regridder(ds, out_grid, 'bilinear', periodic=True)
    ds_blnr = blnr_regridder(ds)
    
    # Conservative
    cons_regridder = xe.Regridder(ds, out_grid, 'conservative', periodic=True)
    ds_cons = cons_regridder(ds)
    
    # Store
    ds_blnr.to_netcdf(out_path + 'regridded/bilinear/' + metric + '/' + model + '.nc')
    ds_blnr.to_netcdf(out_path + 'regridded/conservative/' + metric + '/' + model + '.nc')

In [ ]:
%%time
# Annual avgs
delayed_res = []

for model in models:
    # Check if already exists
    if (os.path.isfile(out_path + 'regridded/bilinear/annual_avgs/' + model + '.nc') and
        os.path.isfile(out_path + 'regridded/conservative/annual_avgs/' + model + '.nc')):
        print(model + ' already done')
        continue
    
    # Dask to parallelize
    delayed_res.append(dask.delayed(regrid)(model, out_grid, out_path, 'annual_avgs'))

# Compute
res = dask.compute(*delayed_res)   

CanESM5 already done
CNRM-CM6-1 already done
CNRM-ESM2-1 already done
EC-Earth3 already done
MIROC6 already done


In [ ]:
%%time
# Annual maxs
delayed_res = []

for model in models:
    # Check if already exists
    if (os.path.isfile(out_path + 'regridded/bilinear/annual_maxs/' + model + '.nc') and
        os.path.isfile(out_path + 'regridded/conservative/annual_maxs/' + model + '.nc')):
        print(model + ' already done')
        continue
    
    # Dask to parallelize
    delayed_res.append(dask.delayed(regrid)(model, out_grid, out_path, 'annual_maxs'))

# Compute
res = dask.compute(*delayed_res)   

## carbonplan

In [2]:
# NEX-GDDP out grid (same as CIL)
out_grid = xr.open_dataset('/gpfs/group/kaf26/default/dcl5300/lafferty-sriver_inprep_tbh_DATA/metrics/cil-gdpcir/annual_avgs/CanESM5.nc')

out_grid = xr.Dataset({'lat': out_grid.lat,
                       'lon': out_grid.lon})

In [3]:
# Set paths
# UPDATE THIS FOR REPRODUCTION
path = '/gpfs/group/kaf26/default/dcl5300/lafferty-sriver_inprep_tbh_DATA/metrics/carbonplan/'

In [4]:
# Regridding function
def regrid(method, model, out_grid, path, metric):
    # Read native grid
    ds = xr.open_zarr(path + 'native_grid/' + method + '/' + metric + '/' + model).load()
    ds = ds.isel(member_id=0) # get rid of member_id
    
    ## xESMF regridder
    # Conservative
    cons_regridder = xe.Regridder(ds, out_grid, 'conservative', periodic=True)
    ds_cons = cons_regridder(ds)
    
    # Store
    ds_cons.to_netcdf(path + 'regridded/conservative/' + method + '/' + metric + '/' + model + '.nc')

### GARD-SV

In [5]:
method = 'GARD-SV'

In [6]:
# Models
from utils import gardsv_ssp_dict

models = list(gardsv_ssp_dict.keys())

In [7]:
%%time
# Annual avgs
for model in models:
    # Check if already exists
    if os.path.isfile(path + 'regridded/conservative/GARD-SV/annual_avgs/' + model + '.nc'):
        print(model + ' already done')
        continue
    
    # Only a few models so no dask
    regrid(method, model, out_grid, path, 'annual_avgs')
    print(model)

BCC-CSM2-MR already done
CanESM5 already done


/storage/home/d/dcl5300/work/ENVS/climate-stack/lib/python3.10/site-packages/xesmf/backend.py:53: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


MIROC6


/storage/home/d/dcl5300/work/ENVS/climate-stack/lib/python3.10/site-packages/xesmf/backend.py:53: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


MPI-ESM1-2-HR
CPU times: user 4min 7s, sys: 40.8 s, total: 4min 47s
Wall time: 4min 58s


In [ ]:
%%time
# Annual maxs
for model in models:
    # Check if already exists
    if os.path.isfile(path + 'regridded/conservative/GARD-SV/annual_maxs/' + model + '.nc'):
        print(model + ' already done')
        continue
    
    # Only a few models so no dask
    regrid(method, model, out_grid, path, 'annual_maxs')
    print(model)